# Comparatif transformer finetuned vs core-news-fr-lg finetuned

Bonus : training sur GPU avec Colab

## Install

In [ ]:
#!pip install -U 'spacy[cuda12x,transformers,lookups]'
# ne pas réinstaller cupy sur Google Colab

In [9]:
!pip install -U 'spacy[transformers,lookups]'

In [10]:
#!python -m spacy download fr_dep_news_trf

In [11]:
!nvidia-smi

Fri Oct 13 20:45:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Prepare training

In [12]:
import spacy

In [3]:
# Création du répertoire si nécessaire
!mkdir -p ./dataset
# Téléchargement du dataset
!wget -O ./dataset/train.spacy https://github.com/jchazalon/hn-ariane-ner-tuto-2023/raw/main/dataset/train.spacy
!wget -O ./dataset/test.spacy https://github.com/jchazalon/hn-ariane-ner-tuto-2023/raw/main/dataset/test.spacy

--2023-10-13 20:59:27--  https://github.com/jchazalon/hn-ariane-ner-tuto-2023/raw/main/dataset/train.spacy
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jchazalon/hn-ariane-ner-tuto-2023/main/dataset/train.spacy [following]
--2023-10-13 20:59:28--  https://raw.githubusercontent.com/jchazalon/hn-ariane-ner-tuto-2023/main/dataset/train.spacy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 817493 (798K) [application/octet-stream]
Saving to: ‘./dataset/train.spacy’

./dataset/train.spa 100%[===================>] 798.33K  --.-KB/s    in 0.02s   

2023-10-13 20:59:28 (39.9 MB/s) - ‘./dataset/train.s

In [11]:
from spacy.tokens import DocBin
test_set = DocBin().from_disk("./dataset/test.spacy")
print(f"Loaded a dataset with {len(test_set)} documents.")

Loaded a dataset with 30 documents.


In [15]:
%%bash
mkdir -p training-scratch
echo '
# -------8<---------8<----------
# This is an auto-generated partial config. To use it with 'spacy train'
# you can run spacy init fill-config to auto-fill all default settings:
# python -m spacy init fill-config ./base_config.cfg ./config.cfg
[paths]
train = null
dev = null
vectors = null
[system]
gpu_allocator = "pytorch"

[nlp]
lang = "fr"
pipeline = ["transformer","ner"]
batch_size = 128

[components]

[components.transformer]
factory = "transformer"

[components.transformer.model]
@architectures = "spacy-transformers.TransformerModel.v3"
name = "camembert-base"
tokenizer_config = {"use_fast": true}

[components.transformer.model.get_spans]
@span_getters = "spacy-transformers.strided_spans.v1"
window = 128
stride = 96

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0

[components.ner.model.tok2vec.pooling]
@layers = "reduce_mean.v1"

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0

[training]
accumulate_gradient = 3
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"

[training.optimizer]
@optimizers = "Adam.v1"

[training.optimizer.learn_rate]
@schedules = "warmup_linear.v1"
warmup_steps = 250
total_steps = 20000
initial_rate = 5e-5

[training.batcher]
@batchers = "spacy.batch_by_padded.v1"
discard_oversize = true
size = 2000
buffer = 256

[initialize]
vectors = ${paths.vectors}
# -------8<---------8<----------
' > ./training-scratch/base_config.cfg

In [16]:
# TEACHER
!python -m spacy init fill-config ./training-scratch/base_config.cfg ./training-scratch/config.cfg


2023-10-13 20:45:27.575333: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
training-scratch/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


## 🚧 Launch training

In [ ]:
!python -m spacy train ./training-scratch/config.cfg --gpu-id 0 --output ./training-scratch/output-trf --paths.train ./dataset/train.spacy --paths.dev ./dataset/test.spacy


2023-10-13 20:01:28.143858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: training-scratch/output-trf
ℹ Saving to output directory: training-scratch/output-trf
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        4595.08   1134.14    0.08    0.05    0.37    0.00
  5     200      490927.29  63053.45   51.51   56.64   47.23    0.52
 10     400        3198.61  12553.72   83.68   81.00   86.53    0.84
 15     600         991.01   9653.16   82.52   80.63   84.50    0.83
 20     800         401.60   9034.19   81.87   78.61   85.42    0.82
 25    1000         23

récupération du modèle après crash VM Colab

In [ ]:
!wget -O training-scratch/model-best.tar.gz https://xxxxxxxxxxx/model-best.tar.gz

In [20]:
!tar -C training-scratch -xzf training-scratch/model-best.tar.gz

## Chargement du modèle entraîné, évaluation et utilisation


In [9]:
from typing import Iterable
from spacy import Language
from spacy.tokens import Doc
from spacy.training import Example
from spacy.scorer import Scorer
def evaluate(ner_model: Language, dataset: Iterable[Doc], debug: bool=False) -> dict:
    examples = []
    for doc_ref in dataset.get_docs(ner_model.vocab):
        text = doc_ref.text
        # doc_pred = None
        # with ner_model.select_pipes(enable="ner"):  # does not work with trained model
        #     doc_pred = ner_model(text)
        doc_pred = ner_model(text)
        if debug:
            print("Pred.:", [(ent.text, ent.label_) for ent in doc_pred.ents], " ↔ Targ.:", [(ent.text, ent.label_) for ent in doc_ref.ents])
        example = Example(doc_pred, doc_ref)
        examples.append(example)

    scorer = Scorer()
    scores = scorer.score_spans(examples, "ents")
    # print(scores["ents_f"])
    return scores

In [22]:
my_model = spacy.load("./training-scratch/output-trf/model-best")
evaluate(my_model, test_set)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'ents_p': 0.8389982110912343,
 'ents_r': 0.8653136531365314,
 'ents_f': 0.8519527702089009,
 'ents_per_type': {'PER': {'p': 0.8835227272727273,
   'r': 0.933933933933934,
   'f': 0.908029197080292},
  'LOC': {'p': 0.7632850241545893,
   'r': 0.7559808612440191,
   'f': 0.7596153846153845}}}

C'est mieux que le modèle large de base de spaCy sur nos documents (fscore 68% sur loc), mais est-ce mieux que le modèle large fine-tuné ?

## Try finetuning fr_core_news_lg

In [1]:
%%bash
mkdir -p training-lg
echo '
# -------8<---------8<----------
# This is an auto-generated partial config. To use it with 'spacy train'
# you can run spacy init fill-config to auto-fill all default settings:
# python -m spacy init fill-config ./base_config.cfg ./config.cfg
[paths]
train = null
dev = null
vectors = "fr_core_news_lg"
[system]
gpu_allocator = null

[nlp]
lang = "fr"
pipeline = ["tok2vec","ner"]
batch_size = 1000

[components]

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${components.tok2vec.model.encode.width}
attrs = ["NORM", "PREFIX", "SUFFIX", "SHAPE"]
rows = [5000, 1000, 2500, 2500]
include_static_vectors = true

[components.tok2vec.model.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 256
depth = 8
window_size = 1
maxout_pieces = 3

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0

[training]
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"

[training.optimizer]
@optimizers = "Adam.v1"

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2

[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001

[initialize]
vectors = ${paths.vectors}
# -------8<---------8<----------
' > ./training-lg/base_config.cfg

In [2]:
!python -m spacy init fill-config ./training-lg/base_config.cfg ./training-lg/config.cfg

2023-10-13 20:58:56.807043: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 20:58:57.828528: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-13 20:58:59.185813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-13 20:58:59.186254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [5]:
!python -m spacy download fr_core_news_lg

2023-10-13 21:00:34.449156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 21:00:35.483504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-13 21:00:36.806468: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-13 21:00:36.806888: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [6]:
!python -m spacy train ./training-lg/config.cfg --gpu-id 0 --output ./training-lg/output-trf --paths.train ./dataset/train.spacy --paths.dev ./dataset/test.spacy


2023-10-13 21:01:09.590052: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 21:01:10.705141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-13 21:01:12.158550: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-13 21:01:12.159086: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [8]:
import spacy

In [12]:
my_model = spacy.load("./training-lg/output-trf/model-best")
evaluate(my_model, test_set)

{'ents_p': 0.7183600713012478,
 'ents_r': 0.7435424354243543,
 'ents_f': 0.7307343608340888,
 'ents_per_type': {'PER': {'p': 0.7154046997389034,
   'r': 0.8228228228228228,
   'f': 0.7653631284916201},
  'LOC': {'p': 0.7247191011235955,
   'r': 0.6172248803827751,
   'f': 0.6666666666666667}}}

Semble moins performant que le modèle de base, et surtout moins performant que le modèle transformer !